In [5]:
!pip install git+https://github.com/shahashka/SP-GIES

  Cloning https://github.com/shahashka/SP-GIES to /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-y57jj4y7
  Running command git clone --filter=blob:none --quiet https://github.com/shahashka/SP-GIES /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-y57jj4y7
  Resolved https://github.com/shahashka/SP-GIES to commit b8302f9d0b5d995e0e371f5c7c61609b437fe939
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... - error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [65 lines of output]
      /private/var/folders/hr/9_dgjtbx6ll7bkdgtm_4dkl00000gn/T/pip-req-build-y57jj4y7/setup.py:9: SetuptoolsDeprecationWarning: As setuptools moves its configuration towards `pyproject.toml`,
      `setuptools.config.read_configuration` became deprecated.
      
      For the time being, you can use the `setuptools.config.setupcfg` module
      to ac